In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
from scipy import stats
from tempfile import mkdtemp
from shutil import rmtree

import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
import xgboost as xgb

from sklearn import set_config
set_config(display = 'diagram')

# Sklearn preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE

In [3]:
train_df= pd.read_csv("raw_data/churn_df.csv")

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 44 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  2501 non-null   int64  
 1   msno                        2501 non-null   object 
 2   is_churn                    2501 non-null   int64  
 3   payment_method_id           2501 non-null   float64
 4   payment_plan_days           2501 non-null   float64
 5   plan_list_price             2501 non-null   float64
 6   actual_amount_paid          2501 non-null   float64
 7   is_auto_renew               2501 non-null   float64
 8   is_cancel                   2501 non-null   float64
 9   remaining_plan_duration     2501 non-null   int64  
 10  is_discount                 2501 non-null   int64  
 11  num_25                      2501 non-null   float64
 12  num_50                      2501 non-null   float64
 13  num_75                      2501 

In [5]:
train_df = train_df.drop(['Unnamed: 0', 'usage_from_ltd', 'msno','discount_percentage'], axis=1)

In [6]:
train_df.isnull().sum()

is_churn                         0
payment_method_id                0
payment_plan_days                0
plan_list_price                  0
actual_amount_paid               0
is_auto_renew                    0
is_cancel                        0
remaining_plan_duration          0
is_discount                      0
num_25                           0
num_50                           0
num_75                           0
num_985                          0
num_100                          0
num_unq                          0
total_secs                       0
city                             0
bd                            1094
registered_via                   0
period_0_churn                   0
period_-1_churn                  0
period_-2_churn                  0
period_-3_churn                  0
period_-4_churn                  0
period_-5_churn                  0
last_transaction_year            0
last_transaction_month_sin       0
last_transaction_month_cos       0
last_transaction_day

In [7]:
train_df.is_churn.value_counts()

is_churn
0    2193
1     308
Name: count, dtype: int64

In [8]:
scale_features = ['payment_method_id',
                  'payment_plan_days',
                  'plan_list_price',
                  'actual_amount_paid',
                  'remaining_plan_duration',
                  'num_25',
                  'num_50',
                  'num_75',
                  'num_985',
                  'num_100',
                  'num_unq',
                  'total_secs',
                  'city',
                  'bd',
                  'registered_via']

In [9]:
# Build the pipeline with the different steps
num_pipeline = make_pipeline(SimpleImputer(strategy="mean"),
                             StandardScaler())

preprocessor = ColumnTransformer([
    ('num_transformer', num_pipeline, scale_features)],
    remainder='passthrough'
)


In [10]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['payment_method_id', 'payment_plan_days',
                                  'plan_list_price', 'actual_amount_paid',
                                  'remaining_plan_duration', 'num_25', 'num_50',
                                  'num_75', 'num_985', 'num_100', 'num_unq',
                                  'total_secs', 'city', 'bd',
                                  'registered_via'])])

In [11]:
X = train_df.drop(['is_churn'], axis=1)
y= train_df['is_churn']

In [12]:
preprocessor.fit(X)
            

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['payment_method_id', 'payment_plan_days',
                                  'plan_list_price', 'actual_amount_paid',
                                  'remaining_plan_duration', 'num_25', 'num_50',
                                  'num_75', 'num_985', 'num_100', 'num_unq',
                                  'total_secs', 'city', 'bd',
                                  'registered_via'])])

In [13]:
X_train_transformed = preprocessor.fit_transform(X)

In [14]:
X_train_transformed = pd.DataFrame(
    X_train_transformed, 
    columns=preprocessor.get_feature_names_out()
)

In [15]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 39 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   num_transformer__payment_method_id        2501 non-null   float64
 1   num_transformer__payment_plan_days        2501 non-null   float64
 2   num_transformer__plan_list_price          2501 non-null   float64
 3   num_transformer__actual_amount_paid       2501 non-null   float64
 4   num_transformer__remaining_plan_duration  2501 non-null   float64
 5   num_transformer__num_25                   2501 non-null   float64
 6   num_transformer__num_50                   2501 non-null   float64
 7   num_transformer__num_75                   2501 non-null   float64
 8   num_transformer__num_985                  2501 non-null   float64
 9   num_transformer__num_100                  2501 non-null   float64
 10  num_transformer__num_unq            

In [16]:
X_train_transformed.head()

,num_transformer__payment_method_id,num_transformer__payment_plan_days,num_transformer__plan_list_price,num_transformer__actual_amount_paid,num_transformer__remaining_plan_duration,num_transformer__num_25,num_transformer__num_50,num_transformer__num_75,num_transformer__num_985,num_transformer__num_100,...,remainder__expire_year,remainder__expire_month_sin,remainder__expire_month_cos,remainder__expire_day_sin,remainder__expire_day_cos,remainder__registration_year,remainder__registration_month_sin,remainder__registration_month_cos,remainder__registration_day_sin,remainder__registration_day_cos
0,0.149706,-0.182971,-0.122138,-0.119878,-0.198484,-0.196405,1.322492,0.430585,-0.538058,-0.721550,...,2017.0,8.660254e-01,-5.000000e-01,0.937752,0.347305,2014.0,-0.866025,0.500000,-0.201299,0.979530
1,0.612517,-0.182971,-0.122138,-0.119878,0.366535,0.086998,1.068319,-0.062272,-0.113284,0.123547,...,2017.0,1.224647e-16,-1.000000e+00,0.998717,-0.050649,2014.0,-0.500000,0.866025,0.571268,0.820763
2,-1.470134,6.674005,5.260570,5.257813,2.671454,2.141674,1.576665,0.923442,2.860132,-0.131948,...,2018.0,8.660254e-01,5.000000e-01,-0.848644,0.528964,2014.0,-0.500000,0.866025,0.651372,-0.758758
3,-1.470134,2.933836,1.859887,1.860299,1.084019,-0.408958,-0.202546,-0.062272,0.736263,-0.407096,...,2017.0,-1.000000e+00,-1.836970e-16,0.848644,0.528964,2016.0,-0.500000,0.866025,-0.651372,-0.758758
4,0.612517,-0.182971,-0.224568,-0.222213,1.003302,-0.408958,-0.456719,-0.555129,-0.538058,-0.092641,...,2017.0,-5.000000e-01,-8.660254e-01,-0.897805,-0.440394,2010.0,-0.500000,-0.866025,-0.201299,0.979530


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_train_transformed, y, test_size=0.2, random_state=42)

In [18]:
print("Before SMOTE, counts of label '1':", sum(y_train==1))
print("Before SMOTE, counts of label '0':", sum(y_train==0))

# Apply SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Check class distribution in resampled dataset
print("After SMOTE, counts of label '1':", sum(y_train_res==1))
print("After SMOTE, counts of label '0':", sum(y_train_res==0))

Before SMOTE, counts of label '1': 248
Before SMOTE, counts of label '0': 1752
After SMOTE, counts of label '1': 1752
After SMOTE, counts of label '0': 1752


In [19]:
#X_train_transformed.replace([np.inf, -np.inf], np.nan, inplace=True)

In [20]:
#X_train_transformed.isnull().sum()

In [21]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from sklearn.model_selection import cross_val_score

def get_estimators():
    return {
        'LogisticRegression': LogisticRegression(solver='lbfgs', max_iter=1000),
        'KNN': KNeighborsClassifier(),
        'RandomForest': RandomForestClassifier(n_estimators=100),
        'XGB': XGBClassifier(), 
        'Vaoting Classifier': VotingClassifier(estimators=[('gbr', GradientBoostingClassifier()), ('xgb', XGBClassifier())]),
        'AdaBoostClassifier': AdaBoostClassifier(base_estimator=GradientBoostingClassifier(),n_estimators=100)
    }



In [22]:
# cross-validation for each model and store results in a dict
results = []
estimators = get_estimators()
for name, estimator in estimators.items():
    scores = cross_val_score(estimator, X_train_res,y_train_res, cv=5)
    results.append({
        'Model': name,
        'Mean Accuracy': np.mean(scores),
    })

/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer(estimator, *args, **routed_params.get(name).score)
  File "/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sk

In [23]:
results

[{'Model': 'LogisticRegression', 'Mean Accuracy': 0.8430409618911758},
 {'Model': 'KNN', 'Mean Accuracy': nan},
 {'Model': 'RandomForest', 'Mean Accuracy': 0.9743207662522927},
 {'Model': 'XGB', 'Mean Accuracy': 0.9688974933768086},
 {'Model': 'Vaoting Classifier', 'Mean Accuracy': 0.9646174852251885},
 {'Model': 'AdaBoostClassifier', 'Mean Accuracy': 0.9703248420623598}]

In [27]:
print(X_train_res.shape)
print(y_train_res.shape)
print(X_test.shape)
print(y_test.shape)

(3504, 39)
(3504,)
(501, 39)
(501,)


In [40]:
type(X_train)

pandas.core.frame.DataFrame

In [29]:
result=[]
for name, estimator in estimators.items():
    estimator.fit(X_train_res, y_train_res)
    if name == 'KNN':
        y_pred = estimator.predict(X_test.values)
    else:
        y_pred = estimator.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    result.append(f"Accuracy of {name} is: {accuracy:.2f}")
result

/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/nazneen/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.

['Accuracy of LogisticRegression is: 0.87',
 'Accuracy of KNN is: 0.80',
 'Accuracy of RandomForest is: 0.97',
 'Accuracy of XGB is: 0.96',
 'Accuracy of Vaoting Classifier is: 0.96',
 'Accuracy of AdaBoostClassifier is: 0.96']

In [63]:
preproc_selector = SelectPercentile(
    mutual_info_regression,
    percentile=25, 
)

In [64]:
preproc_selector.fit(X_train_res, y_train_res)

SelectPercentile(percentile=25,
                 score_func=<function mutual_info_regression at 0x7f56f0195d80>)

In [65]:
selected_features_mask = preproc_selector.get_support()


In [66]:
X_train_res.select_dtypes(include='float').columns[selected_features_mask]

Index(['num_transformer__actual_amount_paid',
       'num_transformer__remaining_plan_duration', 'num_transformer__num_25',
       'num_transformer__num_100', 'remainder__last_transaction_day_sin',
       'remainder__last_transaction_day_cos', 'remainder__expire_day_sin',
       'remainder__expire_day_cos', 'remainder__registration_day_sin',
       'remainder__registration_day_cos'],
      dtype='object')

In [44]:
X_train_res

,num_transformer__payment_method_id,num_transformer__payment_plan_days,num_transformer__plan_list_price,num_transformer__actual_amount_paid,num_transformer__remaining_plan_duration,num_transformer__num_25,num_transformer__num_50,num_transformer__num_75,num_transformer__num_985,num_transformer__num_100,...,remainder__expire_year,remainder__expire_month_sin,remainder__expire_month_cos,remainder__expire_day_sin,remainder__expire_day_cos,remainder__registration_year,remainder__registration_month_sin,remainder__registration_month_cos,remainder__registration_day_sin,remainder__registration_day_cos
0,0.149706,-0.182971,-0.122138,-0.119878,-0.207453,-0.408958,-0.202546,-0.062272,-0.113284,0.162854,...,2017.000000,0.866025,-5.000000e-01,0.848644,0.528964,2010.000000,-8.660254e-01,-0.500000,-0.937752,0.347305
1,-0.313105,-0.182971,-0.122138,-0.119878,-0.261264,-0.550660,-0.456719,-0.555129,-0.113284,-0.682244,...,2017.000000,0.866025,-5.000000e-01,0.937752,0.347305,2012.000000,-2.449294e-16,1.000000,-0.790776,-0.612106
2,0.381112,-0.182971,-0.122138,-0.119878,-0.207453,3.416991,-0.202546,0.923442,-0.113284,-0.092641,...,2017.000000,0.866025,-5.000000e-01,0.937752,0.347305,2011.000000,8.660254e-01,-0.500000,0.651372,-0.758758
3,-1.007322,-0.182971,-0.122138,-0.119878,-0.261264,-0.338107,-0.202546,-0.555129,-0.538058,-0.721550,...,2017.000000,0.866025,-5.000000e-01,-0.201299,0.979530,2012.000000,8.660254e-01,0.500000,-0.485302,-0.874347
4,0.149706,-0.182971,-0.122138,-0.119878,-0.171579,-0.408958,-0.202546,-0.555129,-0.538058,-0.682244,...,2017.000000,0.866025,-5.000000e-01,0.968077,-0.250653,2006.000000,-8.660254e-01,0.500000,-0.299363,-0.954139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,-1.394422,-0.467832,-0.482884,-0.480288,1.478497,0.718208,-0.096249,-0.295263,0.512295,1.446048,...,2017.472735,0.929360,-2.636326e-01,-0.285464,-0.338901,2015.418204,4.094004e-01,0.763633,0.601152,-0.338901
3500,0.402766,-0.182971,-0.122138,-0.119878,0.021172,0.091548,-0.432933,-0.555129,-0.498308,-0.006563,...,2017.000000,0.831773,-5.342523e-01,0.910953,0.243801,2010.093579,8.660254e-01,-0.406421,-0.934014,0.273593
3501,0.612517,-0.182971,-0.138924,-0.136649,0.216530,-0.077926,-0.160892,1.004212,-0.538058,-0.323354,...,2017.000000,0.336119,-8.660254e-01,-0.759693,0.632975,2010.836119,7.541785e-01,0.141925,0.672797,0.183528
3502,-3.998917,6.674005,5.260570,5.257813,2.706045,0.729801,0.432955,0.606624,0.948764,-0.296184,...,2018.000000,0.866025,5.000000e-01,-0.670342,0.716538,2014.000000,-1.785905e-01,-0.333491,-0.312867,0.069014


In [46]:
y_train

2123    0
2280    0
1085    0
1868    0
1412    0
       ..
1638    0
1095    0
1130    0
1294    0
860     0
Name: is_churn, Length: 2000, dtype: int64

In [48]:
new_data = pd.merge(X_train_res,y_train_res, left_index=True, right_index=True)

In [51]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 40 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   num_transformer__payment_method_id        3504 non-null   float64
 1   num_transformer__payment_plan_days        3504 non-null   float64
 2   num_transformer__plan_list_price          3504 non-null   float64
 3   num_transformer__actual_amount_paid       3504 non-null   float64
 4   num_transformer__remaining_plan_duration  3504 non-null   float64
 5   num_transformer__num_25                   3504 non-null   float64
 6   num_transformer__num_50                   3504 non-null   float64
 7   num_transformer__num_75                   3504 non-null   float64
 8   num_transformer__num_985                  3504 non-null   float64
 9   num_transformer__num_100                  3504 non-null   float64
 10  num_transformer__num_unq            

In [68]:
# nn= new_data[['is_churn','num_transformer__actual_amount_paid',
#        'num_transformer__remaining_plan_duration', 'num_transformer__num_25',
#        'num_transformer__num_100', 'remainder__last_transaction_day_sin',
#        'remainder__last_transaction_day_cos', 'remainder__expire_day_sin',
#        'remainder__expire_day_cos', 'remainder__registration_day_sin',
#        'remainder__registration_day_cos']]
# corr_matrix = nn.corr()
# plt.figure(figsize=(8, 6))
# heatmap = sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5, linecolor='black')
# plt.title('Correlation Heatmap')
# plt.show()